In [12]:
import numpy as np
import os
import graphviz
import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import tree
from graphviz import Source
from IPython.display import SVG
from IPython.display import display

In [21]:
def partition(x):
    map = {}
    for i in range(len(x)):
        val = x[i]
        if map.get(val) != None:
            map[val].append(i)
        else:
            map[val] = [i];
    return map


def entropy(y):
    map = partition(y)
    res = 0
    for key in map.keys():
        res -= len(map.get(key))/len(y) * math.log2(len(map.get(key))/len(y))
    return res


def mutual_information(x, y):
    res = entropy(y)
    map = partition(x)
    for list in map.values():
        curr = []
        for val in list:
            curr.append(y[val])
        res -= len(curr)/len(y) * entropy(curr)
    return res


def id3(x, y, attribute_value_pairs=None, depth=0, max_depth=5):
    treeNode = {}
    if attribute_value_pairs == None:
        attribute_value_pairs = []
        for row in x:
            for i in range(len(row)):
                if (i, row[i]) not in attribute_value_pairs:
                    attribute_value_pairs.append((i, row[i]))
    if len(y) == 0:
        return None
    count = 0
    for val in y:
        if val == 1:
            count += 1
    if count == len(y):
        return 1
    if count == 0:
        return 0
    if depth == max_depth or len(attribute_value_pairs) == 0:
        return 1 if count * 2 >= len(y) else 0
    max_pair = ()
    max_gain = -1
    for pair in attribute_value_pairs:
        curr_gain = mutual_information(column(x, pair[0]), y)
        if curr_gain > max_gain:
            max_gain = curr_gain
            max_pair = pair

    attribute_value_pairs.remove(max_pair)
    x_true, y_true, x_false, y_false = data_split(x, y, max_pair)
    treeNode.update({(max_pair[0], max_pair[1], True): id3(x_true, y_true, attribute_value_pairs, depth+1, max_depth)})
    treeNode.update({(max_pair[0], max_pair[1], False): id3(x_false, y_false, attribute_value_pairs, depth+1, max_depth)})
    return treeNode

def column(matrix, i):
    return [row[i] for row in matrix]


def data_split(x, y, pair):
    x_true = []
    y_true = []
    x_false = []
    y_false = []
    for i in range(len(x)):
        if (x[i][pair[0]] == pair[1]):
            x_true.append(x[i])
            y_true.append(y[i])
        else:
            x_false.append(x[i])
            y_false.append(y[i])
    return x_true, y_true, x_false, y_false


def predict_example(x, tree):
    if type(tree) == dict:
        for key in tree:
            if (x[key[0]] == key[1] and key[2] == True) or (x[key[0]] != key[1] and key[2] == False):
                return predict_example(x, tree.get(key))
    return tree
    raise Exception('Function not yet implemented!')


def compute_error(y_true, y_pred):
    sum = 0
    for i in range(len(y_true)):
        if (y_true[i] != y_pred[i]):
            sum += 1
    return sum / len(y_true)
    raise Exception('Function not yet implemented!')


def pretty_print(tree, depth=0):
    """
    Pretty prints the decision tree to the console. Use print(tree) to print the raw nested dictionary representation
    DO NOT MODIFY THIS FUNCTION!
    """
    if depth == 0:
        print('TREE')

    for index, split_criterion in enumerate(tree):
        sub_trees = tree[split_criterion]

        # Print the current node: split criterion
        print('|\t' * depth, end='')
        print('+-- [SPLIT: x{0} = {1} {2}]'.format(split_criterion[0], split_criterion[1], split_criterion[2]))

        # Print the children
        if type(sub_trees) is dict:
            pretty_print(sub_trees, depth + 1)
        else:
            print('|\t' * (depth + 1), end='')
            print('+-- [LABEL = {0}]'.format(sub_trees))


def render_dot_file(dot_string, save_file, image_format='png'):
    """
    Uses GraphViz to render a dot file. The dot file can be generated using
        * sklearn.tree.export_graphviz()' for decision trees produced by scikit-learn
        * to_graphviz() (function is in this file) for decision trees produced by  your code.
    DO NOT MODIFY THIS FUNCTION!
    """
    if type(dot_string).__name__ != 'str':
        raise TypeError('visualize() requires a string representation of a decision tree.\nUse tree.export_graphviz()'
                        'for decision trees produced by scikit-learn and to_graphviz() for decision trees produced by'
                        'your code.\n')

    # Set path to your GraphViz executable here
    os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
    graph = graphviz.Source(dot_string)
    graph.format = image_format
    graph.render(save_file, view=True)


def to_graphviz(tree, dot_string='', uid=-1, depth=0):
    """
    Converts a tree to DOT format for use with visualize/GraphViz
    DO NOT MODIFY THIS FUNCTION!
    """

    uid += 1       # Running index of node ids across recursion
    node_id = uid  # Node id of this node

    if depth == 0:
        dot_string += 'digraph TREE {\n'

    for split_criterion in tree:
        sub_trees = tree[split_criterion]
        attribute_index = split_criterion[0]
        attribute_value = split_criterion[1]
        split_decision = split_criterion[2]

        if not split_decision:
            # Alphabetically, False comes first
            dot_string += '    node{0} [label="x{1} = {2}?"];\n'.format(node_id, attribute_index, attribute_value)

        if type(sub_trees) is dict:
            if not split_decision:
                dot_string, right_child, uid = to_graphviz(sub_trees, dot_string=dot_string, uid=uid, depth=depth + 1)
                dot_string += '    node{0} -> node{1} [label="False"];\n'.format(node_id, right_child)
            else:
                dot_string, left_child, uid = to_graphviz(sub_trees, dot_string=dot_string, uid=uid, depth=depth + 1)
                dot_string += '    node{0} -> node{1} [label="True"];\n'.format(node_id, left_child)

        else:
            uid += 1
            dot_string += '    node{0} [label="y = {1}"];\n'.format(uid, sub_trees)
            if not split_decision:
                dot_string += '    node{0} -> node{1} [label="False"];\n'.format(node_id, uid)
            else:
                dot_string += '    node{0} -> node{1} [label="True"];\n'.format(node_id, uid)

    if depth == 0:
        dot_string += '}\n'
        return dot_string
    else:
        return dot_string, node_id, uid


# if __name__ == '__main__':
# Load the training data
M = np.genfromtxt('./monks-1.train', missing_values=0, skip_header=0, delimiter=',', dtype=int)
ytrn = M[:, 0]
Xtrn = M[:, 1:]

# Load the test data
M = np.genfromtxt('./monks-1.test', missing_values=0, skip_header=0, delimiter=',', dtype=int)
ytst = M[:, 0]
Xtst = M[:, 1:]

# Learn a decision tree of depth 3
decision_tree = id3(Xtrn, ytrn, max_depth=3)

# Pretty print it to console
pretty_print(decision_tree)

# Visualize the tree and save it as a PNG image
# dot_str = to_graphviz(decision_tree)
# render_dot_file(dot_str, './my_learned_tree')

# Compute the test error
y_pred = [predict_example(x, decision_tree) for x in Xtst]
tst_err = compute_error(ytst, y_pred)

print('Test Error = {0:4.2f}%.'.format(tst_err * 100))

# # question b
# for i in range(1, 4):
#     trainFileName = "./monks_data/monks-" + str(i) + ".train"
#     testFileName = "./monks_data/monks-" + str(i) + ".test"
#     # Load the training data
#     M = np.genfromtxt(trainFileName, missing_values=0, skip_header=0, delimiter=',', dtype=int)
#     ytrn = M[:, 0]
#     Xtrn = M[:, 1:]

#     # Load the test data
#     M = np.genfromtxt(testFileName, missing_values=0, skip_header=0, delimiter=',', dtype=int)
#     ytst = M[:, 0]
#     Xtst = M[:, 1:]

#     tst_errs = []
#     trn_errs = []
#     for depth in range(1, 11):
#         decision_tree = id3(Xtrn, ytrn, max_depth=depth)
#         y_pred_tst = [predict_example(x, decision_tree) for x in Xtst]
#         y_pred_trn = [predict_example(x, decision_tree) for x in Xtrn]
#         tst_err = compute_error(ytst, y_pred_tst)
#         trn_err = compute_error(ytrn, y_pred_trn)
#         tst_errs.append(tst_err)
#         trn_errs.append(trn_err)
#     plt.figure()
#     plt.title("monks-" + str(i))
#     plt.plot(range(1, 11), tst_errs, marker='o', linewidth=3, markersize=12)
#     plt.plot(range(1, 11), trn_errs, marker='s', linewidth=3, markersize=12)
#     plt.xlabel("depth", fontsize=12)
#     plt.ylabel("error", fontsize=12)
#     plt.legend(["test error", "training error"], fontsize=16)
#     plt.xticks(range(1, 11), fontsize=12)
#     plt.axis([0, 11, 0, 1])

# # question c
# M = np.genfromtxt("./monks_data/monks-1.train", missing_values=0, skip_header=0, delimiter=',', dtype=int)
# ytrn = M[:, 0]
# Xtrn = M[:, 1:]

# # Load the test data
# M = np.genfromtxt("./monks_data/monks-1.test", missing_values=0, skip_header=0, delimiter=',', dtype=int)
# ytst = M[:, 0]
# Xtst = M[:, 1:]
# for depth in [1, 3, 5]:
#     decision_tree = id3(Xtrn, ytrn, max_depth=depth)
#     dot_str = to_graphviz(decision_tree)
#     render_dot_file(dot_str, './my_learned_tree_dep' + str(depth))
#     y_pred = [predict_example(x, decision_tree) for x in Xtst]
#     print("depth=" + str(depth))
#     print(confusion_matrix(ytst, y_pred))

# # question d
# M = np.genfromtxt("./monks_data/monks-1.train", missing_values=0, skip_header=0, delimiter=',', dtype=int)
# ytrn = M[:, 0]
# Xtrn = M[:, 1:]

# # Load the test data
# M = np.genfromtxt("./monks_data/monks-1.test", missing_values=0, skip_header=0, delimiter=',', dtype=int)
# ytst = M[:, 0]
# Xtst = M[:, 1:]
# for depth in [1, 3, 5]:
#     decision_tree = tree.DecisionTreeClassifier(criterion="entropy", max_depth=depth)
#     decision_tree = decision_tree.fit(Xtrn, ytrn)

    graph = Source(tree.export_graphviz(decision_tree, out_file=None, class_names=['0', '1'] , filled = True))
    display(SVG(graph.pipe(format='svg')))

    y_pred = decision_tree.predict(Xtst)
    print('depth=' + str(depth))
    print(confusion_matrix(ytst, y_pred))

# question e
M = np.genfromtxt("./balance-scale.data", missing_values=0, skip_header=0, delimiter=',', dtype=str)
M2 = np.ones((576, 5))
idx = 0
for row in M:
    if row[0] == 'L':
        M2[idx][0] = 0
        for i in range(1, len(row)):
            M2[idx][i] = int(row[i])
        idx += 1
    if row[0] == 'R':
        M2[idx][0] = 1
        for i in range(1, len(row)):
            M2[idx][i] = int(row[i])
        idx += 1
ytrn = M2[:400, 0]
Xtrn = M2[:400, 1:]
ytst = M2[401:, 0]
Xtst = M2[401:, 1:]
for depth in [1, 3, 5]:
    decision_tree = id3(Xtrn, ytrn, max_depth=depth)
    dot_str = to_graphviz(decision_tree)
    render_dot_file(dot_str, './my_learned_tree_e_dep' + str(depth))
    y_pred = [predict_example(x, decision_tree) for x in Xtst]
    print("depth=" + str(depth))
    print(confusion_matrix(ytst, y_pred))
for depth in [1, 3, 5]:
    decision_tree = tree.DecisionTreeClassifier(criterion="entropy", max_depth=depth)
    decision_tree = decision_tree.fit(Xtrn, ytrn)

    graph = Source(tree.export_graphviz(decision_tree, out_file=None, class_names=['0', '1'] , filled = True))
    display(SVG(graph.pipe(format='svg')))

    y_pred = decision_tree.predict(Xtst)
    print('depth=' + str(depth))
    print(confusion_matrix(ytst, y_pred))
    
    

TREE
+-- [SPLIT: x4 = 3 True]
|	+-- [SPLIT: x5 = 1 True]
|	|	+-- [SPLIT: x3 = 1 True]
|	|	|	+-- [LABEL = 0]
|	|	+-- [SPLIT: x3 = 1 False]
|	|	|	+-- [LABEL = 0]
|	+-- [SPLIT: x5 = 1 False]
|	|	+-- [SPLIT: x2 = 1 True]
|	|	|	+-- [LABEL = 1]
|	|	+-- [SPLIT: x2 = 1 False]
|	|	|	+-- [LABEL = 0]
+-- [SPLIT: x4 = 3 False]
|	+-- [SPLIT: x4 = 2 True]
|	|	+-- [SPLIT: x3 = 3 True]
|	|	|	+-- [LABEL = 0]
|	|	+-- [SPLIT: x3 = 3 False]
|	|	|	+-- [LABEL = 0]
|	+-- [SPLIT: x4 = 2 False]
|	|	+-- [SPLIT: x4 = 4 True]
|	|	|	+-- [LABEL = 0]
|	|	+-- [SPLIT: x4 = 4 False]
|	|	|	+-- [LABEL = 1]
Test Error = 27.08%.
